# Libraries


In [ ]:
import yfinance as yf
import pandas as pd
from yahooquery import Ticker, search
from yfinance.screener.query import EquityQuery
from yfinance.screener.screener import screen
import sys
import sqlite3
import time

# Populating ticker csv file from downloaded sources

In [ ]:
'''# Load NASDAQ tickers
nasdaq = pd.read_csv("nasdaqlisted.csv")
nasdaq_sample = nasdaq.sample(n=1000, random_state=42)
nasdaq_sample = nasdaq_sample[["Symbol"]].rename(columns={"Symbol": "ticker"})

# Load NYSE/other tickers
other = pd.read_csv("otherlisted.csv")
other_sample = other.sample(n=1000, random_state=42)
other_sample = other_sample[["ACT Symbol"]].rename(columns={"ACT Symbol": "ticker"})

# Combine into one DataFrame
combined = pd.concat([nasdaq_sample, other_sample], ignore_index=True)

# Save to CSV
combined.to_csv("tickers.csv", index=False)'''

print("Saved 2000 combined tickers to 'tickers.csv'")

'''
# Load the file
df = pd.read_csv("tickers.csv")

# Print all tickers
for ticker in df["ticker"]:
    print(ticker)
'''

# Database Initiation

In [ ]:
def init_database(db_path="tickers.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # --- Create tickers table (company metadata) ---
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS tickers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT UNIQUE,
            name TEXT,
            exchange TEXT,
            region TEXT,
            sector TEXT,
            industry TEXT,
            market_cap REAL,
            pe_ratio REAL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)

    # --- Create stock_prices table (live price data) ---
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS stock_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT UNIQUE,
            price REAL,
            change REAL,
            change_percent REAL,
            volume INTEGER,
            last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY(symbol) REFERENCES tickers(symbol)
        )
    """)

    conn.commit()
    conn.close()
    
init_database()

# Functions for ticker Database:


In [ ]:
def ticker_exists(cursor, symbol):
    cursor.execute("SELECT 1 FROM tickers WHERE symbol = ?", (symbol,))
    return cursor.fetchone() is not None

def insert_ticker(cursor, symbol, info=None):
    try:
        # Fetch info only if not provided
        if info is None:
            info = yf.Ticker(symbol).info

        cursor.execute("""
            INSERT INTO tickers (
                symbol, name, exchange, region, sector, industry,
                market_cap, pe_ratio
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            symbol,
            info.get("shortName"),
            info.get("exchange"),
            info.get("region", "Unknown"),
            info.get("sector"),
            info.get("industry"),
            info.get("marketCap"),
            info.get("trailingPE")
        ))

        return True

    except Exception as e:
        print(f"Failed ticker: {symbol}: {e}")
        return False

def insert_tickers_batch(tickers, db_path="tickers.db", start=0, limit=None, delay=1.0):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    end = start + limit if limit else len(tickers)
    batch = tickers[start:end]

    # Filter out tickers that already exist
    batch = [symbol for symbol in batch if not ticker_exists(cursor, symbol)]

    print(f"{len(batch)} new tickers to insert into tickers")

    for i, symbol in enumerate(batch, start=start + 1):
        success = insert_ticker(cursor, symbol)
        if success:
            print(f"[{i}] Inserted ticker: {symbol}")
        time.sleep(delay)

    conn.commit()
    conn.close()


# Functions for Prices Database


In [ ]:
def price_exists(cursor, symbol):
    cursor.execute("SELECT 1 FROM stock_prices WHERE symbol = ?", (symbol,))
    return cursor.fetchone() is not None

def insert_price(cursor, symbol, info=None):
    try:
        # Fetch info only if not provided
        if info is None:
            info = yf.Ticker(symbol).info

        cursor.execute("""
            INSERT INTO stock_prices (
                symbol, price, change, change_percent, volume
            ) VALUES (?, ?, ?, ?, ?)
        """, (
            symbol,
            info.get("regularMarketPrice"),
            info.get("regularMarketChange"),
            info.get("regularMarketChangePercent"),
            info.get("volume")
        ))

        return True

    except Exception as e:
        print(f"✖ Failed price: {symbol}: {e}")
        return False

def insert_prices_batch(tickers, db_path="tickers.db", start=0, limit=None, delay=1.0):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    end = start + limit if limit else len(tickers)
    batch = tickers[start:end]

    # Pre-filter to only tickers not yet in stock_prices
    batch = [symbol for symbol in batch if not price_exists(cursor, symbol)]

    print(f"{len(batch)} new tickers to insert into stock_prices")

    for i, symbol in enumerate(batch, start=start + 1):
        success = insert_price(cursor, symbol)
        if success:
            print(f"[{i}] Inserted price: {symbol}")
        time.sleep(delay)

    conn.commit()
    conn.close()


# Making sure that all tickers are in prices

In [ ]:
def sync_missing_prices(db_path="tickers.db", delay=1.0):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT symbol FROM tickers")
    all_tickers = [row[0] for row in cursor.fetchall()]

    missing = [symbol for symbol in all_tickers if not price_exists(cursor, symbol)]

    print(f"{len(missing)} tickers missing from stock_prices — syncing...")

    for i, symbol in enumerate(missing, 1):
        if insert_price(cursor, symbol):
            print(f"[{i}] Synced price: {symbol}")
        time.sleep(delay)

    conn.commit()
    conn.close()


def sync_and_insert_all(tickers, db_path="tickers.db", delay=1.0):
    # Step 1: Sync existing tickers with missing prices
    sync_missing_prices(db_path=db_path, delay=delay)

    # Step 2: Insert new tickers and/or their prices
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    print(f"Checking {len(tickers)} input tickers for insertion...")

    for i, symbol in enumerate(tickers, 1):
        in_tickers = ticker_exists(cursor, symbol)
        in_prices = price_exists(cursor, symbol)

        if in_tickers and in_prices:
            continue

        try:
            info = yf.Ticker(symbol).info

            if not in_tickers:
                insert_ticker(cursor, symbol, info=info)

            if not in_prices:
                insert_price(cursor, symbol, info=info)

            print(f"[{i}] Inserted: {symbol}")

        except Exception as e:
            print(f"[{i}] Failed: {symbol} - {e}")

        time.sleep(delay)

    conn.commit()
    conn.close()
    
sync_missing_prices()


In [ ]:
def load_tickers(csv_path="tickers.csv"):
    df = pd.read_csv(csv_path)
    return df["ticker"].dropna().unique().tolist()

tickers = load_tickers("tickers.csv")
sync_and_insert_all(tickers, db_path="tickers.db", delay=0.7)

# Initiating Stock Price Hourly and Daily

In [ ]:
def init_history_tables(db_path="tickers.db"):
    conn = sqlite3.connect(db_path, timeout=10)
    cursor = conn.cursor()

    # Daily candles
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS prices_daily (
            symbol TEXT,
            date DATE,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume INTEGER,
            PRIMARY KEY (symbol, date)
        )
    """)

    # Hourly candles
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS prices_hourly (
            symbol TEXT,
            timestamp DATETIME,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume INTEGER,
            PRIMARY KEY (symbol, timestamp)
        )
    """)

    conn.commit()
    conn.close()


In [ ]:
def get_all_symbols(db_path="tickers.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT symbol FROM tickers")
    symbols = [row[0] for row in cursor.fetchall()]
    conn.close()
    return symbols


In [ ]:
def daily_data_exists(cursor, symbol):
    cursor.execute("SELECT 1 FROM prices_daily WHERE symbol = ? LIMIT 1", (symbol,))
    return cursor.fetchone() is not None

def hourly_data_exists(cursor, symbol):
    cursor.execute("SELECT 1 FROM prices_hourly WHERE symbol = ? LIMIT 1", (symbol,))
    return cursor.fetchone() is not None

def insert_historical_prices(symbol, table, db_path="tickers.db", period="5y", interval="1d", delay=1.0):
    conn = sqlite3.connect(db_path, timeout=10)
    cursor = conn.cursor()

    # 1. Check if data already exists
    key_column = "date" if interval == "1d" else "timestamp"
    cursor.execute(f"SELECT 1 FROM {table} WHERE symbol = ? LIMIT 1", (symbol,))
    if cursor.fetchone():
        conn.commit()
        conn.close()
        return

    try:
        # 2. Download data
        try:
            data = yf.download(symbol, period=period, interval=interval, progress=False, auto_adjust=True)
        except Exception as e:
            print(f"Download failed for {symbol}: {e}")
            conn.close()
            return

        if data.empty:
            print(f"No data for {symbol}")
            conn.close()
            return

        # 3. Flatten if needed
        if isinstance(data.columns, pd.MultiIndex):
            data = data.xs(symbol, level=1, axis=1)

        # 4. Insert rows
        for idx, row in data.iterrows():
            timestamp = idx.strftime("%Y-%m-%d %H:%M:%S") if interval != "1d" else idx.date()
            cursor.execute(f"""
                INSERT OR IGNORE INTO {table} (
                    symbol, "{key_column}",
                    open, high, low, close, volume
                ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                symbol,
                timestamp,
                row.get("Open"),
                row.get("High"),
                row.get("Low"),
                row.get("Close"),
                int(row["Volume"]) if not pd.isna(row["Volume"]) else None
            ))

        print(f"Inserted {len(data)} rows for {symbol}")

        conn.commit()

    except Exception as e:
        print(f"Failed to process {symbol}: {e}")

    finally:
        conn.close()
        time.sleep(delay)



In [ ]:
symbols = get_all_symbols()

for symbol in symbols:
    insert_historical_prices(
        symbol=symbol,
        table="prices_daily",
        period="5y",
        interval="1d",
        delay=0.5
    )

for symbol in symbols:
    insert_historical_prices(
        symbol=symbol,
        table="prices_hourly",
        period="7d",
        interval="1h",
        delay=0.5
    )


# Testing Area / Junkyard

In [ ]:

def print_hourly_prices(db_path="tickers.db", limit=None):
    conn = sqlite3.connect(db_path, timeout=10)

    query = "SELECT * FROM prices_hourly"
    if limit:
        query += f" LIMIT {limit}"

    try:
        df = pd.read_sql_query(query, conn)
        print(df)
        print(f"\nTotal rows: {len(df)}")
    except Exception as e:
        print(f"Error reading hourly prices: {e}")
    finally:
        conn.close()


print_hourly_prices()           # Print all
print_hourly_prices(limit=50)   # Print first 50 rows

data = yf.download("AAL", period="7d", interval="1h")

print(data)


In [ ]:
def test_download_tsla_hourly():
    symbol = "TSLA"
    period = "7d"
    interval = "1h"

    data = yf.download(symbol, period=period, interval=interval, progress=False, auto_adjust=True)

    # Flatten multi-index if needed
    if isinstance(data.columns, pd.MultiIndex):
        data = data.xs(symbol, level=1, axis=1)

    if data is None or data.empty:
        print("No data returned for TSLA.")
        return

    print(f"Downloaded {len(data)} rows for {symbol}:")
    print(data.head())

# Run the test
test_download_tsla_hourly()